In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from tensorflow.keras.layers import (Dense, 
                                     BatchNormalization, 
                                     LeakyReLU, 
                                     Reshape, 
                                     Conv2DTranspose,
                                     Conv2D,
                                     Dropout,
                                     Flatten)

import tensorflow_addons as tfa
import tensorflow_datasets as tfds

from tensorflow.keras.applications import EfficientNetB3
from keras.models import Model
import efficientnet.tfkeras
import efficientnet.keras as efn 

from gen_disc import *

tfds.disable_progress_bar()


In [ ]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
dataset, metadata = tfds.load('cycle_gan/horse2zebra',
                              with_info=True, as_supervised=True)

train_pepsi, train_coke = dataset['trainA'], dataset['trainB']
test_pepsi, test_coke = dataset['testA'], dataset['testB']

In [ ]:
#loading in all datasets as numpy arrays and labels from files created by 'createdataset.py'
load_train_pepsi = np.load('datasets/train_pepsi.npy')
load_train_coke = np.load('datasets/train_coke.npy')
load_test_pepsi = np.load('datasets/test_pepsi.npy')
load_test_coke = np.load('datasets/test_coke.npy')

load_train_pepsi_label = np.load('datasets/train_pepsi_label.npy')
load_train_coke_label = np.load('datasets/train_coke_label.npy')
load_test_pepsi_label = np.load('datasets/test_pepsi_label.npy')
load_test_coke_label = np.load('datasets/test_coke_label.npy')


In [ ]:
print(load_train_pepsi.shape)
print(load_train_pepsi_label.shape)

print(load_train_pepsi_label[3])

In [ ]:
#creating tensorflow datasets with numpy arrays
train_pepsi = tf.data.Dataset.from_tensor_slices((load_train_pepsi, load_train_pepsi_label))
train_coke = tf.data.Dataset.from_tensor_slices((load_train_coke, load_train_coke_label))
test_pepsi = tf.data.Dataset.from_tensor_slices((load_test_pepsi, load_test_pepsi_label))
test_coke = tf.data.Dataset.from_tensor_slices((load_test_coke, load_test_coke_label))

In [ ]:
print(train_pepsi)

In [ ]:
# Define the standard image size.
orig_img_size = (286, 286)
# Size of the random crops to be used during training.
input_img_size = (256, 256, 3)
# Weights initializer for the layers.
kernel_init = keras.initializers.RandomNormal(mean=0.0, stddev=0.02)
# Gamma initializer for instance normalization.
gamma_init = keras.initializers.RandomNormal(mean=0.0, stddev=0.02)

buffer_size = 256
batch_size = 1


def normalize_img(img):
    img = tf.cast(img, dtype=tf.float32)
    # Map values in the range [-1, 1]
    return (img / 127.5) - 1.0


def preprocess_train_image(img, label):
    # Random flip
    img = tf.image.random_flip_left_right(img)
    # Resize to the original size first
    img = tf.image.resize(img, [*orig_img_size])
    # Random crop to 256X256
    img = tf.image.random_crop(img, size=[*input_img_size])
    # Normalize the pixel values in the range [-1, 1]
    img = normalize_img(img)
    return img

def preprocess_test_image(img, label):
    # Only resizing and normalization for the test images.
    img = tf.image.resize(img, [input_img_size[0], input_img_size[1]])
    img = normalize_img(img)
    return img

In [ ]:
autotune = tf.data.experimental.AUTOTUNE
# Apply the preprocessing operations to the training data
train_pepsi = (train_pepsi.map(preprocess_train_image, num_parallel_calls=autotune).cache().shuffle(buffer_size).batch(batch_size))
train_coke = (train_coke.map(preprocess_train_image, num_parallel_calls=autotune).cache().shuffle(buffer_size).batch(batch_size))

# Apply the preprocessing operations to the test data
test_pepsi = (test_pepsi.map(preprocess_test_image, num_parallel_calls=autotune).cache().shuffle(buffer_size).batch(batch_size))
test_coke = (test_coke.map(preprocess_test_image, num_parallel_calls=autotune).cache().shuffle(buffer_size).batch(batch_size))


In [ ]:
_, ax = plt.subplots(4, 2, figsize=(10, 15))
for i, samples in enumerate(zip(train_pepsi.take(4), train_coke.take(4))):
    pepsi = (((samples[0][0] * 127.5) + 127.5).numpy()).astype(np.uint8)
    coke = (((samples[1][0] * 127.5) + 127.5).numpy()).astype(np.uint8)
    ax[i, 0].imshow(pepsi)
    ax[i, 1].imshow(coke)
plt.show()

!!!
Model Compiling
!!!

In [ ]:
sample_pepsi = next(iter(train_pepsi))
sample_coke = next(iter(train_coke))


#sample_pepsi = train_pepsi.__iter__().next()
#sample_coke = train_coke.as_numpy_iterator().next()

In [ ]:
generator_g = efficientnet_generator()
generator_f = efficientnet_generator()

In [ ]:
generator_g = efficientnet_generator(
    filters = 64,
    name=None,
)
generator_f = efficientnet_generator(
    filters = 64,
    name=None,
)

In [ ]:
OUTPUT_CHANNELS = 3

generator_g = get_resnet_generator(
    filters=64,
    num_downsampling_blocks=2,
    num_residual_blocks=9,
    num_upsample_blocks=2,
    #gamma_initializer=gamma_init,
    name=None,
)

generator_f = get_resnet_generator(
    filters=64,
    num_downsampling_blocks=2,
    num_residual_blocks=9,
    num_upsample_blocks=2,
    #gamma_initializer=gamma_init,
    name=None,
)

In [ ]:
discriminator_x = get_discriminator(name="discriminator_X")
discriminator_y = get_discriminator(name="discriminator_Y")

In [ ]:
to_coke = generator_g(sample_pepsi)
to_pepsi = generator_f(sample_coke)
plt.figure(figsize=(8, 8))
contrast = 8

imgs = [sample_pepsi, to_coke, sample_coke, to_pepsi]
title = ['Pepsi', 'To Coke', 'Coke', 'To Pepsi']

for i in range(len(imgs)):
  plt.subplot(2, 2, i+1)
  plt.title(title[i])
  if i % 2 == 0:
    plt.imshow(imgs[i][0] * 0.5 + 0.5)
  else:
    plt.imshow(imgs[i][0] * 0.5 * contrast + 0.5)
plt.show()

In [ ]:
plt.figure(figsize=(8, 8))

plt.subplot(121)
plt.title('Is a real coke can?')
plt.imshow(discriminator_y(sample_coke)[0, ..., -1], cmap='RdBu_r')

plt.subplot(122)
plt.title('Is a real pepsi can?')
plt.imshow(discriminator_x(sample_pepsi)[0, ..., -1], cmap='RdBu_r')

plt.show()

In [ ]:
LAMBDA = 100

In [ ]:
loss_obj = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [ ]:
def discriminator_loss(real, generated):
  real_loss = loss_obj(tf.ones_like(real), real)

  generated_loss = loss_obj(tf.zeros_like(generated), generated)

  total_disc_loss = real_loss + generated_loss

  return total_disc_loss * 0.5

In [ ]:
def generator_loss(generated):
  return loss_obj(tf.ones_like(generated), generated)

In [ ]:
def calc_cycle_loss(real_image, cycled_image):
  loss1 = tf.reduce_mean(tf.abs(real_image - cycled_image))
  
  return LAMBDA * loss1

In [ ]:
def identity_loss(real_image, same_image):
  loss = tf.reduce_mean(tf.abs(real_image - same_image))
  return LAMBDA * 0.5 * loss

In [ ]:
generator_g_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
generator_f_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

discriminator_x_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
discriminator_y_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

In [ ]:
checkpoint_path = "./checkpoints/train"

ckpt = tf.train.Checkpoint(generator_g=generator_g,
                           generator_f=generator_f,
                           discriminator_x=discriminator_x,
                           discriminator_y=discriminator_y,
                           generator_g_optimizer=generator_g_optimizer,
                           generator_f_optimizer=generator_f_optimizer,
                           discriminator_x_optimizer=discriminator_x_optimizer,
                           discriminator_y_optimizer=discriminator_y_optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

# if a checkpoint exists, restore the latest checkpoint.
if ckpt_manager.latest_checkpoint:
  ckpt.restore(ckpt_manager.latest_checkpoint)
  print ('Latest checkpoint restored!!')

In [ ]:
EPOCHS = 50

In [ ]:
def generate_images(model, test_input):
  prediction = model(test_input)
    
  plt.figure(figsize=(12, 12))

  display_list = [test_input[0], prediction[0]]
  title = ['Input Image', 'Predicted Image']

  for i in range(2):
    plt.subplot(1, 2, i+1)
    plt.title(title[i])
    # getting the pixel values between [0, 1] to plot it.
    plt.imshow(display_list[i] * 0.5 + 0.5)
    plt.axis('off')
  plt.show()

In [ ]:
@tf.function
def train_step(real_x, real_y):
  # persistent is set to True because the tape is used more than
  # once to calculate the gradients.
  with tf.GradientTape(persistent=True) as tape:
    # Generator G translates X -> Y
    # Generator F translates Y -> X.
    
    fake_y = generator_g(real_x, training=True)
    cycled_x = generator_f(fake_y, training=True)

    fake_x = generator_f(real_y, training=True)
    cycled_y = generator_g(fake_x, training=True)

    # same_x and same_y are used for identity loss.
    same_x = generator_f(real_x, training=True)
    same_y = generator_g(real_y, training=True)

    disc_real_x = discriminator_x(real_x, training=True)
    disc_real_y = discriminator_y(real_y, training=True)

    disc_fake_x = discriminator_x(fake_x, training=True)
    disc_fake_y = discriminator_y(fake_y, training=True)

    # calculate the loss
    gen_g_loss = generator_loss(disc_fake_y)
    gen_f_loss = generator_loss(disc_fake_x)
    
    total_cycle_loss = calc_cycle_loss(real_x, cycled_x) + calc_cycle_loss(real_y, cycled_y)
    
    # Total generator loss = adversarial loss + cycle loss
    total_gen_g_loss = gen_g_loss + total_cycle_loss + identity_loss(real_y, same_y)
    total_gen_f_loss = gen_f_loss + total_cycle_loss + identity_loss(real_x, same_x)

    disc_x_loss = discriminator_loss(disc_real_x, disc_fake_x)
    disc_y_loss = discriminator_loss(disc_real_y, disc_fake_y)
  
  # Calculate the gradients for generator and discriminator
  generator_g_gradients = tape.gradient(total_gen_g_loss, 
                                        generator_g.trainable_variables)
  generator_f_gradients = tape.gradient(total_gen_f_loss, 
                                        generator_f.trainable_variables)
  
  discriminator_x_gradients = tape.gradient(disc_x_loss, 
                                            discriminator_x.trainable_variables)
  discriminator_y_gradients = tape.gradient(disc_y_loss, 
                                            discriminator_y.trainable_variables)
  
  # Apply the gradients to the optimizer
  generator_g_optimizer.apply_gradients(zip(generator_g_gradients, 
                                            generator_g.trainable_variables))

  generator_f_optimizer.apply_gradients(zip(generator_f_gradients, 
                                            generator_f.trainable_variables))
  
  discriminator_x_optimizer.apply_gradients(zip(discriminator_x_gradients,
                                                discriminator_x.trainable_variables))
  
  discriminator_y_optimizer.apply_gradients(zip(discriminator_y_gradients,
                                                discriminator_y.trainable_variables))

In [ ]:
import time
import os
from IPython.display import clear_output

In [ ]:
for epoch in range(EPOCHS):
  start = time.time()

  n = 0
  for image_x, image_y in tf.data.Dataset.zip((train_pepsi, train_coke)):
    train_step(image_x, image_y)
    if n % 10 == 0:
      print ('.', end='')
    n+=1

  clear_output(wait=True)
  # Using a consistent image (sample_pepsi) so that the progress of the model
  # is clearly visible.
  generate_images(generator_g, sample_pepsi)

  if (epoch + 1) % 5 == 0:
    ckpt_save_path = ckpt_manager.save()
    print ('Saving checkpoint for epoch {} at {}'.format(epoch+1,
                                                         ckpt_save_path))

  print ('Time taken for epoch {} is {} sec\n'.format(epoch + 1,
                                                      time.time()-start))
